# Exact Diagonalization of Spin Hamiltonian
    Alan Morningstar
    May 2017

#### specify lattice and Hamiltonian parameters

In [1]:
# NOTE: using periodic boundary conditions
# square lattice length
const Lx = 6;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# plaquette coupling
const K = 0.0;

In [2]:
# container for Hamiltonian couplings
immutable couplings
    # 1st nearest neighbor
    J1::Float64;
    # plaquette terms
    K::Float64;
end;

In [3]:
const c = couplings(J1,K);

## Lattice and Bond Structure

#### useful functions for defining the lattice connectivity

In [4]:
# convert single integer site index to xy indexing
function xyIndex(site::Int64,Lx::Int64,Ly::Int64)
    # site - number of the site for which x,y indices are computed

    y::Int,x::Int = divrem(site-1,Lx);
    
    return x,y;
end;

In [5]:
# convert xy site indexing to single integer index
function siteIndex(xy::Tuple{Int,Int},Lx::Int64,Ly::Int64)
    # xy - tuple containing x,y indexing of a site

    x = xy[1];
    y = xy[2];
    
    # move xy into primary lattice cell
    while x > (Lx-1)
        x -= Lx;
    end;
    while x < 0
        x += Lx;
    end;
    while y > (Ly-1)
        y -= Ly;
    end;
    while y < 0
        y += Ly;
    end;    
    
    # map to integer index
    site::Int64 = y*Lx + x + 1;
    
    return site;
end;

In [6]:
# find indices of neighbors specified by a translation vectors T
function neighbors(site::Int64,T::Array{Tuple{Int64,Int64},1},Lx::Int64,Ly::Int64)
    # use xy indexing just to compute neighbors
    x::Int,y::Int = xyIndex(site,Lx,Ly);

    # neighbor in xy indexing
    neighborxy::Array{Tuple{Int,Int},1} = [(x+t[1],y+t[2]) for t in T];
    
    # back to single integer indexing
    neighborIndex::Array{Int,1} = [siteIndex(nxy,Lx,Ly) for nxy in neighborxy];
    
    # return neighbor index
    return neighborIndex;
end;

#### define the lattice

In [7]:
# plaquette sites corresponding to a given site (p1) in the bottom left of the plaquette
# p3L indicates p3 on the plaquette to the left, similarly for p1D on the plaquette in the downwards direction
#
# p3L--p3--p4
#   |  |Plq|
# p1L--p1--p2
#  |  |   |
#   p1D p2D

In [8]:
# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

In [9]:
# container for lattice properties
immutable lattice
    # size of lattice, number of sites
    Lx::Int64;
    Ly::Int64;
    N::Int64;
    nbrs::Array{Array{Int64,1},1};
    
    function lattice(Lx::Int64,Ly::Int64,neighborVectors::Array{Tuple{Int64,Int64},1})
        neighborsList = [neighbors(site,neighborVectors,Lx,Ly) for site in 1:N];
        new(Lx,Ly,Lx*Ly,neighborsList);
    end;
end;

In [10]:
const l = lattice(Lx,Ly,neighborVectors);

## Build $(S_z,k_x,k_y)$ Symmetry Sector Basis
Note: see Sandvik's Computational Studies of Quantum Spin Systems

#### specify symmetry sector

In [11]:
# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int,N/2);
# choose kx,ky by specifying mi such that ki=2*pi*mi/Li where i=x,y ... mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

In [12]:
# container for symmetry sector
immutable sector
    # number of 1 bits in the bit rep. of spin states in this sector
    n1::Int64;
    # momentum
    kx::Float64;
    ky::Float64;
end;

In [13]:
const s = sector(n1,kx,ky);

#### defining some useful functions for this section

In [14]:
# set a bit to 1
function setBit(i::UInt64,bit::Int64)
    return (i | (1 << UInt64(bit-1)))::UInt64;
end;

# set a bit to 0
function clearBit(i::UInt64,bit::Int64)
    return (i & (~(UInt64(1) << UInt64(bit-1))))::UInt64;
end;

# toggle a bit
function toggleBit(i::UInt64,bit::Int64)
    return (i $ (UInt64(1) << UInt64(bit-1)))::UInt64;
end;

# read a bit
function readBit(i::UInt64,bit::Int64)
    return ((i >> UInt64(bit-1)) & UInt64(1))::UInt64;
end;

# swap two bits
function swapBits(i::UInt64,bit1::Int64,bit2::Int64)
    if readBit(i,bit1) == readBit(i,bit2)
        return i::UInt64;
    else
        return (i $ ( (UInt64(1)<<UInt64(bit1-1)) | (UInt64(1)<<UInt64(bit2-1)) ))::UInt64;
    end;
end;

In [15]:
# x translation operator
function Tx(b::UInt64,l::lattice)
    # loop over rows of the lattice
    for y::Int in 0:l.Ly-1
        # roll the row
        for x::Int in l.Lx-1:-1:1
            b = swapBits(b,y*l.Lx+1+x,y*l.Lx+x);
        end;
    end;
    
    return b::UInt64;
end;

# y translation operator
function Ty(b::UInt64,l::lattice)
    # loop over columns of the lattice
    for x::Int in 0:l.Lx-1
        # roll the column
        for y::Int in l.Ly-1:-1:1
            b = swapBits(b,y*l.Lx+x+1,(y-1)*l.Lx+x+1);
        end;
    end;
    
    return b::UInt64;
end;

In [16]:
# count number of unique elements in an array
function numUnique(Tbs::Array{UInt64,1})
    # num unique elements
    num::Int64 = 0;
    # scan over elements looking in previous elements
    for i in 1:length(Tbs)
        if !(Tbs[i] in Tbs[1:i-1])
            num += 1;
        end;
    end;
    
    return num::Int64;
end;

In [17]:
# function for computing the normalization constant Na such that the momentum state is 1/sqrt(Na) * sum(phase * TxTy |b>)
function normConstant(b::UInt64,l::lattice,s::sector)
    
    # sum of phases
    F::Complex128 = 0.0+0.0im;
    # normalization constant
    Na::Float64 = 0.0;
    # currrent translated state
    Tb::UInt64 = b;
    # set of translated states in integer rep.
    Tbs::Array{UInt64,1} = Array{UInt64}(l.N);
    
    # perform all translations
    for y in 0:l.Ly-1
        for x in 0:l.Lx-1
            # add to set of translated states
            Tbs[y*Lx+x+1] = Tb;
            
            if Tb == b
                # add to sum of phases
                F += exp(-1.0im*(s.kx*x+s.ky*y));
            elseif Tb < b
                return 0.0
            end;
            
            Tb = Tx(Tb,l);

        end;
        
        Tb = Ty(Tb,l);
        
    end;
    
    # compute and return normalization constant
    Na = abs2(F)*numUnique(Tbs);
    
    return Na;
end;

#### construct the basis

In [18]:
immutable SzkxkyBasis
    # list of representatives of momentum basis states in integer representation
    b::Array{UInt64};
    # list of corresponding normalization constants
    n::Array{Float64};
    # dimension of Hilbert space
    dim::Int64;
    
    # constructor
    function SzkxkyBasis(l::lattice,s::sector)
        # initialize list of reps of momentum basis elements
        bList::Array{UInt64,1} = UInt64[];
        # initialize list of normalization constants
        nList::Array{Float64,1} = Float64[];

        # run up the binary odometer of Sz states
        # start with first n1 bits in state 1(down)
        b::UInt64 = 2^n1-1;

        while true
            # check if this state meets the required kx,ky
            n::Float64 = normConstant(b,l,s);

            # if valid rep state, add info to basis
            if n > 0.0
                push!(bList,b);
                push!(nList,n);
            end;

            # counter of 1 bits to the right (in binary convention)
            i::Int = 0;
            # position in bit array
            bit::Int = 1;
            while bit < l.N
                # find a 1 bit whose following neighbor is 0
                if readBit(b,bit) == 0x1
                    if readBit(b,bit+1) == 0x1
                        i += 1;
                    else
                        # shuffle bits over
                        for J in 1:i
                            b = setBit(b,J);
                        end;
                        for J in i+1:bit
                            b = clearBit(b,J);
                        end;
                        b = setBit(b,bit+1);
                        # then break to next loop
                        break
                    end;
                end;
                # to the next bit
                bit += 1;
            end;

            # if all 1s got shifted completely to the left, then all states are explored
            if bit == l.N
                break
            end;

        end;
        
        # construct basis
        new(bList,nList,length(bList));
    end;
    
end;

In [19]:
# construct the Sz,kx,ky sector basis
# 2.65 seconds for 6x4 lattice, basis is ~0.0002 GB
@time const basis = SzkxkyBasis(l,s);
print("Dimension of reduced Hilbert space is ",basis.dim,".");

  2.704715 seconds (5.56 M allocations: 1.306 GB, 1.93% gc time)
Dimension of reduced Hilbert space is 112800.

In [20]:
# using ProfileView;

In [21]:
# Profile.clear();
# @profile SzkxkyBasis(l,s);
# ProfileView.view()

#### some more useful functions for working with this basis

In [22]:
# search ordered basis for index of integer representation of spin state
function basisIndex(b::UInt64,basis::SzkxkyBasis)
    bIndex::UnitRange{Int64} = searchsorted(basis.b,b)::UnitRange{Int64};
    if !isempty(bIndex)
        return bIndex[1]::Int64;
    else
        return 0::Int64;
    end;
end;

In [23]:
# find related representative state and what translation relates the two states
function representative(b::UInt64,l::lattice)
    # rep. state
    rep::UInt64 = b;
    # translated state
    Tb::UInt64 = b;
    # translations to get to rep. state
    lx::Int64 = 0;
    ly::Int64 = 0;
    
    # perform all translations
    for y::Int in 0:l.Ly-1
        for x::Int in 0:l.Lx-1
            
            if Tb < rep
                # then this is a better representative
                rep = Tb;
                lx = x;
                ly = y;
            end;
            
            Tb = Tx(Tb,l);
        end;
        Tb = Ty(Tb,l);
    end;
    
    return rep::UInt64,lx::Int64,ly::Int64;
end;

## Sparse Hamiltonian
$H=\frac{1}{4} J_1 \sum_{\langle ij \rangle} \bf{\sigma}_i \cdot \bf{\sigma}_j + \frac{1}{8} K \sum_{p} \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_2} \right) \left( \bf{\sigma}_{p_3} \cdot \bf{\sigma}_{p_4} \right) + \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_3} \right) \left( \bf{\sigma}_{p_2} \cdot \bf{\sigma}_{p_4} \right) - \left( \bf{\sigma}_{p_1} \cdot \bf{\sigma}_{p_4} \right) \left( \bf{\sigma}_{p_2} \cdot \bf{\sigma}_{p_3} \right)$

#### useful functions for computing states and matrix elements

In [24]:
# function for flipping two spins of a basis state
function XiXj(b::UInt64,i::Int64,j::Int64)
    # b - integer rep of state
    # i,j - sites of spin flips
    return b $ ( (UInt64(1)<<UInt64(i-1)) | (UInt64(1)<<UInt64(j-1)) );
end;

# function for flipping four spins of a basis state
function XiXjXkXl(b::UInt64,i::Int,j::Int,k::Int,l::Int)
    # b - integer rep of state
    # i,j,k,l - sites of spin flips
    return b $ ( (UInt64(1)<<UInt64(i-1)) | (UInt64(1)<<UInt64(j-1)) | (UInt64(1)<<UInt64(k-1)) | (UInt64(1)<<UInt64(l-1)) );
end;

# function for computing (-1)^(positive integer)
function simplePower(i::UInt64)
    if readBit(i,1) == UInt64(0)
        return 1::Int64;
    else
        return -1::Int64;
    end;
end;

#### construct the sparse Hamiltonian

In [25]:
# function for building the sparse Hamiltonian
function constructSparseHam(basis::SzkxkyBasis,c::couplings,s::sector,l::lattice)
    
    # couplings
    J1::Float64 = c.J1;
    K::Float64 = c.K;
    # momentum
    kx::Float64 = s.kx;
    ky::Float64 = s.ky;
    
    # store location and value of non-zero matrix elements
    I::Array{Int64} = Int64[];
    J::Array{Int64} = Int64[];
    M::Array{Complex128} = Complex128[];
    
    # loop over basis
    for bIndex::Int64 in 1:basis.dim
        
        # spin states of basis
        b::UInt64 = basis.b[bIndex]; # integer rep.
        sb::Array{UInt64,1} = [readBit(b,bit) for bit in 1:l.N]; # spin array rep.
        # normalization constant
        nb::Float64 = basis.n[bIndex];
        
        # initialize the diagonal matrix element
        Hbb::Complex128 = 0.0 + 0.0im;

        # loop over lattice sites
        for site::Int64 in 1:N
            
            # spins at nearby sites p1,p2,p3,p4,p1D,p2D,p1L,p3L
            p::Array{Int64,1} = l.nbrs[site];
            sp::Array{UInt64,1} = sb[p];
            # some common factors in the matrix elements
            sPw12::Int64 = simplePower(sp[1] + sp[2]);
            sPw34::Int64 = simplePower(sp[3] + sp[4]);
            sPw13::Int64 = simplePower(sp[1] + sp[3]);
            sPw24::Int64 = simplePower(sp[2] + sp[4]);
            sPw14::Int64 = simplePower(sp[1] + sp[4]);
            sPw23::Int64 = simplePower(sp[2] + sp[3]);
            sPw1234::Int64 = simplePower(sp[1] + sp[2] + sp[3] + sp[4]);
            sPw1D2D::Int64 = simplePower(sp[5] + sp[6]);
            sPw1L3L::Int64 = simplePower(sp[7] + sp[8]);
            
            # contribute to the diagonal matrix element
            # -----------------------------------------
            Hbb += 0.25*J1*( sPw12 + sPw13 );
            Hbb += 0.125*K*sPw1234;
            
            # compute off diagonal matrix elements
            # ------------------------------------
            
            # the 12 term
            if sPw12 == -1
                # the bra
                a12::UInt64 = XiXj(b,p[1],p[2]);
                # the rep and transaltion of the bra
                a12rep::UInt64,lx12::Int64,ly12::Int64 = representative(a12,l);
                # search for this rep in the basis
                a12repIndex::Int64 = basisIndex(a12rep,basis);
                # the matrix element
                Hsite12::Complex128 = 0.5*J1+0.125*K*(sPw34-sPw1234+2*sPw1D2D)*exp(-1.0im*(kx*lx12+ky*ly12))*sqrt(basis.n[a12repIndex]/nb);
                
                push!(I,bIndex);
                push!(J,a12repIndex);
                push!(M,Hsite12);
            end;
                
            # the 13 term
            if sPw13 == -1
                # the bra
                a13::UInt64 = XiXj(b,p[1],p[3]);
                # the rep and transaltion of the bra
                a13rep::UInt64,lx13::Int64,ly13::Int64 = representative(a13,l);
                # search for this rep in the basis
                a13repIndex::Int64 = basisIndex(a13rep,basis);
                # the matrix element
                Hsite13::Complex128 = 0.5*J1*+0.125*K*(sPw24-sPw1234+2*sPw1L3L)*exp(-1.0im*(kx*lx13+ky*ly13))*sqrt(basis.n[a13repIndex]/nb);
                
                push!(I,bIndex);
                push!(J,a13repIndex);
                push!(M,Hsite13);
            end;
                
            # the 14 term
            if sPw14 == -1
                # the bra
                a14::UInt64 = XiXj(b,p[1],p[4]);
                # the rep and transaltion of the bra
                a14rep::UInt64,lx14::Int64,ly14::Int64 = representative(a14,l);
                # search for this rep in the basis
                a14repIndex::Int64 = basisIndex(a14rep,basis);
                # the matrix element
                Hsite14::Complex128 = -0.250*K*sPw23*exp(-1.0im*(kx*lx14+ky*ly14))*sqrt(basis.n[a14repIndex]/nb);
            
                push!(I,bIndex);
                push!(J,a14repIndex);
                push!(M,Hsite14);
            end;
                
            # the 23 term
            if sPw23 == -1
                # the bra
                a23::UInt64 = XiXj(b,p[2],p[3]);
                # the rep and transaltion of the bra
                a23rep::UInt64,lx23::Int64,ly23::Int64 = representative(a23,l);
                # search for this rep in the basis
                a23repIndex::Int64 = basisIndex(a23rep,basis);
                # the matrix element
                Hsite23::Complex128 = -0.25*K*sPw14*exp(-1.0im*(kx*lx23+ky*ly23))*sqrt(basis.n[a23repIndex]/nb);
           
                push!(I,bIndex);
                push!(J,a23repIndex);
                push!(M,Hsite23);
            end;
                
            # the 1234 term
            if sp[1] + sp[2] + sp[3] + sp[4] == 2
                # the bra
                a1234::UInt64 = XiXjXkXl(b,p[1],p[2],p[3],p[4]);
                # the rep and transaltion of the bra
                a1234rep::UInt64,lx1234::Int64,ly1234::Int64 = representative(a1234,l);
                # search for this rep in the basis
                a1234repIndex::Int64 = basisIndex(a1234rep,basis);
                # the matrix element
                Hsite1234::Complex128 = 0.125*K*(2-sPw12-sPw34-sPw13-sPw24+sPw14+sPw23)*exp(-1.0im*(kx*lx1234+ky*ly1234))*sqrt(basis.n[a1234repIndex]/nb);
            
                push!(I,bIndex);
                push!(J,a1234repIndex);
                push!(M,Hsite1234);
            end;

        end;
        
        # push diagonal matrix element to list of matrix elements
        push!(I,bIndex);
        push!(J,bIndex);
        push!(M,Hbb);
        
    end;
    
    H::SparseMatrixCSC{Complex,Int64} = sparse(I,J,M,basis.dim,basis.dim);
    return H;
end;

In [26]:
# build the sparse Hamiltonian
# takes 237 seconds for 6x4 J1-only Heisenberg model, H contains ~0.06 GB
@time const H = constructSparseHam(basis,c,s,l);

 39.808238 seconds (35.26 M allocations: 1.517 GB, 1.12% gc time)


In [27]:
# using ProfileView

In [28]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

## Compute Eigenvalues

In [29]:
# parameters for eigenvalue calculation
numEigs = 4;
tolerance = 10.^(-5.);
ritzVec = false;
numKrylovVecs = 20;
maxIter = 300;

In [30]:
# ~2.46 seconds for 6x4, numEigs=4, which=:SR, J1 Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);  #:LM stands for largest magnitude, :SR for smallest real part
;

  8.037815 seconds (31.41 M allocations: 726.363 MB, 2.50% gc time)


In [31]:
# energies
print(real(eigsResult[1]));

[-13.958,-11.6926,-11.2511,-11.2063]

In [32]:
# algorithm performance
print("Number of iterations = ",eigsResult[3],"\n");
print("Number of matrix-vector multiplications = ",eigsResult[4],"\n");

Number of iterations = 4
Number of matrix-vector multiplications = 65
